In [1]:
import numpy as np
from astropy import units as u
from astropy import time

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.twobody.propagation import propagate

from poliastro.twobody.propagation import cowell
from poliastro.plotting import OrbitPlotter3D
from poliastro.util import norm

from perylune import geom
from perylune import orbit_tools
from perylune.orbit_tools import *

In [2]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [3]:
# Let's get the initial PW-Sat2 orbit. Its norad id is 43814

# PW-SAT2 TLE data from 2020-march-15
#PW-SAT2                 
#1 43814U 18099BJ  20049.76768004  .00014282  00000-0  95610-3 0  9991
#2 43814  97.7801 128.6236 0010365 203.2339 156.8317 15.06987254 65660

# Needed to handle TLE into Poliastro's Orbit
from tletools import TLE

tle_text = """PW-SAT2                 
1 43814U 18099BJ  20049.76768004  .00014282  00000-0  95610-3 0  9991
2 43814  97.7801 128.6236 0010365 203.2339 156.8317 15.06987254 65660"""
tle_lines = tle_text.strip().splitlines()
tle_pwsat2 = TLE.from_lines(*tle_lines)
orb1 = tle_pwsat2.to_orbit()

print_orb(orb1)

6916 x 6931 km x 97.8 deg (GCRS) orbit around Earth (♁) at epoch 2020-02-18T18:25:27.555456000 (UTC)
a(𝑎)=6923.5493km, b=6923.5456km, e=0.00, i=97.78deg raan(Ω)=128.62deg argp(𝜔)=203.23deg nu(𝜈)=156.88deg
period=5733.29s perapis=6916.3730km(538.24km) apoapsis=6930.7256km(552.59km)


In [18]:
# Rotate the orbit plane by 60 degrees, so it's more "edge on" towards the Sun
orb2 = Orbit.from_classical(Earth, orb1.a, orb1.ecc, orb1.inc, orb1.raan + 60*u.deg, orb1.argp, orb1.nu, orb1.epoch)
print_orb(orb1)
print_orb(orb2)

6916 x 6931 km x 97.8 deg (GCRS) orbit around Earth (♁) at epoch 2020-02-18T18:25:27.555456000 (UTC)
a(𝑎)=6923.5493km, b=6923.5456km, e=0.00, i=97.78deg raan(Ω)=128.62deg argp(𝜔)=203.23deg nu(𝜈)=156.88deg
period=5733.29s perapis=6916.3730km(538.24km) apoapsis=6930.7256km(552.59km)
6916 x 6931 km x 97.8 deg (GCRS) orbit around Earth (♁) at epoch 2020-02-18T18:25:27.555456000 (UTC)
a(𝑎)=6923.5493km, b=6923.5456km, e=0.00, i=97.78deg raan(Ω)=188.62deg argp(𝜔)=203.23deg nu(𝜈)=156.88deg
period=5733.29s perapis=6916.3730km(538.24km) apoapsis=6930.7256km(552.59km)


In [5]:
# These are small utility functions to draw lots of points.
draw_x = []
draw_y = []
draw_z = []
draw_c = [] # color
draw_t = [] # text

decimation_ratio = 10 # Specifies if a given point should be recorded or not.
enable_drawing = True



cnt = 0

def clear_samples():
    global draw_x, draw_y, draw_z, draw_c, draw_t
    draw_x, draw_y, draw_z, draw_c, draw_t = [], [], [], [], []

def record_point(coords, text, color):
    global cnt
    cnt = cnt + 1
    if cnt < decimation_ratio: # Should we remember this point or not
        return
    
    cnt = 0
    x, y, z = coords
    draw_x.append(x)
    draw_y.append(y)
    draw_z.append(z)
    draw_c.append(color)
    draw_t.append(text)


def draw_points(frame):
    
    trace = Scatter3d(
        x=draw_x,
        y=draw_y,
        z=draw_z,
        name="propagation",
        #line=dict(color=color, width=1, dash="solid"),
        mode="markers",  # Boilerplate
        text=draw_t,
        marker=dict(size=2, color=draw_c,               # set color to an array/list of desired values
        #colorscale='Viridis',   # choose a colorscale
        #opacity=0.8
        )
    )
    frame._figure.add_trace(trace)




In [6]:
# Perturber 1: This is a constant forward acceleration. This could model an ion engine,
# but it's very simplified this more more like a theoretical model.
#
# Based on poliastro example.
accel = 2e-6

def pert_constant_accel(accel):
    def constant_acc(t0, u, k):
        """ This function is called every time the propagation algorithm needs to determine the perturbation.

        t0 - time in seconds since beginning
        u - state vectors, u[0:2] is r (position), u[3:5] - velocity
        k - GM constant, expressed in km3/s2 (so it's 398600.4418 for Earth)
        return 3 values - x,y,z delta (in km)
        """
        
        v = u[3:]
        norm_v = (v[0]**2 + v[1]**2 + v[2]**2)**.5 # expressed in km/s
        return accel * v / norm_v

    return constant_acc

In [7]:
# Perturber 2: sail brake. This slowly slows down, the slowdown is dependent on the altitude
accel2 = -2e-5

def pert_aerobrake(accel):
    def constant_acc(t0, u, k):
        """ My **guesses** at the parameters:
        t0 - time in seconds since beginning
        u - state vectors, u[0:2] is r (position), u[3:5] - velocity
        k - GM constant, expressed in km3/s2 (so it's 398600.4418 for Earth)
        return 3 values - x,y,z delta
        """
        
        v = u[3:]
        norm_v = (v[0]**2 + v[1]**2 + v[2]**2)**.5 # expressed in km/s
        return accel * v / norm_v

    return constant_acc

In [8]:
# Perturber 3: Solar pressure. Uses solar pressure of the Sun. Parameters:

solar_rad_pressure = 4.5e-6 # Pascals, or N/m2, source: https://en.wikipedia.org/wiki/Radiation_pressure#Pressures_of_absorption_and_reflection

area = 4 # sail area in m^2, source: https://pw-sat.pl/wp-content/uploads/2018/11/PW-Sat2-w-liczbach-1.pdf
mass = 2.5 # mass of the sat in kilograms, source: https://pw-sat.pl/wp-content/uploads/2018/11/PW-Sat2-w-liczbach-1.pdf

f = solar_rad_pressure * area # in N (N = kg*m/s^2)

solar_a = f / mass # In m/s^2

# The model used requires the value in km/s^2
# To exaggerate the acceleration by a factor of 1000, comment out this line.
#solar_a = solar_a / 1000 # in km/s^2
solar_a = solar_a / 10

print("Force generated by %f m2 solar sail on %fkg spacecraft generates force %fN, accel of %e m/s" % (area, mass, f, solar_a)) # m/s^2

def pert_solar_sail(accel):

    RE = 6378.137 # Earth radius
    def in_shadow(v):
        """ Determines whether an object is in shadow or sunlit. It assumes the Sun is at [147000000,0,0] position. """

        # TODO: See shadow_function in poliastro.core.perturbations, 
        #       also see Curtis, algorithm 12.3 (pg. 702)

        if v[0] > 0: 
            return False # This spacecraft is above terminator, so it's always lit.
        r = (v[1]**2 + v[2]**2)**.5

        return r < RE # If the spacecraft is farther away from the X axis than the Earth radius, it is above the Earth shadow.

    def velocity_angle(v):


        angle = geom.solar_angle(v,None)
        return angle

    def accel_solar(t0, u, k):
        """ My **guesses** at the parameters:
        t0 - time in seconds since beginning
        u - state vectors, u[0:2] is r (position), u[3:5] - velocity
        k - GM constant, expressed in km3/s2 (so it's 398600.4418 for Earth)
        return 3 values - x,y,z delta
        """
        
        r = u[:3]
        v = u[3:]
        norm_v = (v[0]**2 + v[1]**2 + v[2]**2)**.5 # expressed in km/s

        shad = in_shadow(r)
        angle = velocity_angle(v)

        run_propulsion = angle > 150.0 and not shad
        
        if enable_drawing:
            record_point(r, text=str("angle=%f, inshadow=%s" % (angle, shad)), color = "red" if run_propulsion else "black")
        #if t0 < 86400*2:
        #    return 0,0,0 # start acceleration after 2 days
        
        # Sun is located as [1.49e+9, 0, 0], so the solar flux pushes towards negative x
        return -solar_a if run_propulsion else 0, 0, 0
        #return 0,0,0

    return accel_solar

Force generated by 4.000000 m2 solar sail on 2.500000kg spacecraft generates force 0.000018N, accel of 7.200000e-07 m/s


In [9]:
# Calculate propagation for the 100 orbits and sample 1000 points along the way
times = np.linspace(0, 100 * orb2.period, 1000)
clear_samples()
positions = propagate(
    orb2,
    time.TimeDelta(times),
    method=cowell,
    rtol=1e-11,
    #ad=pert_constant_accel(accel),
    ad=pert_solar_sail(accel)
)

In [10]:
from plotly.graph_objects import Figure, Layout, Scatter, Scatter3d

frame = OrbitPlotter3D()
frame.set_attractor(Earth)

# Let's pretend this is the Sun
frame._draw_sphere(600*u.km, "red", "Sun", center=[14700, 0, 0] * u.km)

if enable_drawing:
    draw_points(frame)
else:
    frame.plot_trajectory(positions, color="green", label="PW-Sat2 (modified)")

#frame.plot(orb1, label="orb1")
frame.plot(orb2, label="orb2")

#frame._draw_point(696*u.km, "red", "X", center=[14700, 0, 0] * u.km)
#frame._draw_point(696*u.km, "yellow", "Y", center=[0, 14700, 0] * u.km)
#frame._draw_point(696*u.km, "green", "Z", center=[0, 0, 14700] * u.km)

frame._layout.width = 1200
frame._layout.height = 800
frame._layout.margin=dict(l=10, r=10, b=10, t=10, pad=4 )
frame.show()

In [11]:
import plotly.express as px
diag1 = px.line(x=times.value, y=positions.norm() - Earth.R, labels={'x': 'time [s]', 'y': 'altitude [km]'})
diag1.update_layout(yaxis=dict(range=[300, 555]))

In [12]:

def moving_avg2(a, n):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [13]:
window_size = 10
positions_avg = moving_avg2(positions.norm(), window_size)
diag2 = px.line(x=times[window_size-1:].value/86400, y=positions_avg - Earth.R, labels={'x': 'time [days]', 'y': 'altitude [km]'})
diag2.update_layout(yaxis=dict(range=[544, 546]))
diag2.show()

In [14]:

d_time = [0]
d_a = [orb2.a.value]
d_e = [orb2.ecc.value]
d_apo = [orb2.r_a.value]
d_per = [orb2.r_p.value]

for d in range(30):
    tof = (24.0 * u.h).to(u.s) # 24h
    orb2 = orb2.propagate(tof, method=cowell, 
            #ad=pert_constant_accel(accel),
            ad=pert_solar_sail(accel), 
            rtol=1e-11)
    d_time.append(d_time[-1] + 1)
    d_a.append(orb2.a.value)
    d_e.append(orb2.ecc.value)
    d_apo.append(orb2.r_a.value)
    d_per.append(orb2.r_p.value)

In [15]:
print_orb(orb2)

6030 x 6921 km x 97.8 deg (GCRS) orbit around Earth (♁) at epoch 2020-03-19T18:25:27.555456000 (UTC)
a(𝑎)=6475.5935km, b=6460.2294km, e=0.07, i=97.79deg raan(Ω)=188.94deg argp(𝜔)=90.82deg nu(𝜈)=150.30deg
period=5185.97s perapis=6029.7830km(-348.35km) apoapsis=6921.4039km(543.27km)


In [16]:
import plotly.graph_objs as go
fig = go.Figure()

r = Earth.R.to(u.km).value

fig.add_trace(go.Line(x = d_time, y = d_a - r, mode="markers", name="semi-major axis"))
fig.add_trace(go.Scatter(x = d_time, y = d_apo - r, mode="markers", name="Apoapsis"))
fig.add_trace(go.Scatter(x = d_time, y = d_per - r, mode="markers", name="Periapsis"))
fig.add_trace(go.Scatter(x = d_time, y = d_e, mode="markers", name="eccentricity"))


#fig.add_trace(go.Scatter(x = nu_tbl, y = r_tbl, mode="markers", name="distance [km]"))

# Update legend to show at the top
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), 
                  xaxis_title="Time [days]", yaxis_title="Altitude [km]", title="Perturbations",
                  margin=dict(l=20, r=20, t=20, b=20))

#fig.update_layout(yaxis=dict(range=[530, 560]))

fig.add_annotation(x=0, y=7.24, text="periapsis")
fig.add_annotation(x=pi, y=7.24, text="apoapsis")

fig.show()


/home/thomson/devel/perylune/venv/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [17]:
print_orb(orb2)

6030 x 6921 km x 97.8 deg (GCRS) orbit around Earth (♁) at epoch 2020-03-19T18:25:27.555456000 (UTC)
a(𝑎)=6475.5935km, b=6460.2294km, e=0.07, i=97.79deg raan(Ω)=188.94deg argp(𝜔)=90.82deg nu(𝜈)=150.30deg
period=5185.97s perapis=6029.7830km(-348.35km) apoapsis=6921.4039km(543.27km)
